In [1]:
from gensim.models.word2vec import Word2VecKeyedVectors
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

D:\ProgramData\Anaconda3\envs\dlnlp\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#导入外部语料库
wv_from_text = Word2VecKeyedVectors.load_word2vec_format("E:\\Tencent_AILab_ChineseEmbedding\\Tencent_AILab_ChineseEmbedding.txt", binary=False)

In [3]:
import re

In [4]:
# 打开文本
source = open(r"data/source.txt","r",encoding="UTF-8")
source_para = source.read()
source.close()
target = open(r"data/target.txt", "r", encoding="UTF-8")
# target = open(r"data/科技企业梯度培育政策.txt", "r", encoding="UTF-8")
target_para = target.read()
target.close()

In [5]:
#文本断句
def cut_sent(para):
    para = re.sub('([。！；？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符 \1及\2防止删掉标点符号
    para = re.sub('([。！？；\?][”’])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.strip()  # 段尾如果有多余的\n就去掉它
    return para.split("\n")

In [6]:
# 对源文本进行分词
print(type(source_para))
source_line_list = []
for line in cut_sent(source_para): #文本断句
    if line != "":
        source_line_list.append(line)
print(source_line_list)

<class 'str'>
['外资', '研发投入', '独角兽', '总部', '工商关系', '税收关系', '瞪羚企业', '知识产权', '世界500强', '工程技术研究中心', '股权融资', '营业收入', '雏鹰企业', '龙头', '科技进步奖', '销售收入', '上市', '新三板', '信用数据']


In [7]:
print(('dd就' in wv_from_text))

False


In [8]:
from pyhanlp import *

In [9]:
HanLP.Config.ShowTermNature = True #显示词性标注

In [10]:
NLPTokenizer = JClass("com.hankcs.hanlp.tokenizer.NLPTokenizer")

In [11]:
CustomDictionary = JClass("com.hankcs.hanlp.dictionary.CustomDictionary")

In [12]:
CustomDictionary.insert("万人计划")

True

In [13]:
NLPTokenizer.ANALYZER.enableCustomDictionaryForcing(True)

In [14]:
StandardTokenizer = JClass("com.hankcs.hanlp.tokenizer.StandardTokenizer")
StandardTokenizer.SEGMENT.enableCustomDictionaryForcing(True)

In [15]:
#对目标文本进行分词
target_line_list = []
word_stop_list = ['p', 'w', 'c','u','r','q','m','f','d'] #依据词性确定停用词标识
for line in cut_sent(target_para): #文本断句
    if line != "":
        target_line_seged = []
        target_line_seged_list = NLPTokenizer.segment(line)
#         target_line_seged_list = StandardTokenizer.segment(line)
        for target_word_item in target_line_seged_list:
            # 删除停用词及不在词典中的词，保证后面调用n_simliarity函数不报错
            if str(target_word_item.nature) not in word_stop_list and (target_word_item.word in wv_from_text):
                target_line_seged.append(target_word_item.word)
    target_line_list.append(target_line_seged) #将形成的分词语句置于列表
print(target_line_list)

[['西安', '国家', '自主', '创新', '示范区', '鼓励', '企业', '自主创新', '政策'], ['实施', '创新', '驱动', '发展', '战略', '完善', '西安高新区', '科技', '创', '新体', '系', '提升', '企业', '自主', '创新能力', '加快', '西安', '国家', '自主', '创新', '示范区', '世界', '一流', '科技', '园区', '建设', '进程', '特制', '定本', '政策'], ['实施', '创新', '驱动', '发展', '战略', '完善', '西安高新区', '科技', '创', '新体', '系', '提升', '企业', '自主', '创新能力', '加快', '西安', '国家', '自主', '创新', '示范区', '世界', '一流', '科技', '园区', '建设', '进程', '特制', '定本', '政策'], ['政策', '支持', '对象', '满足', '基本', '条件'], ['工商', '注册', '财', '税关', '系', '高新区', '无', '不良', '信用', '记录', '上报', '统计', '报表', '无', '重大', '安全事故', '发生', '无', '不良', '经营', '行为', '独立', '法人', '企业', '机构'], ['工商', '注册', '财', '税关', '系', '高新区', '无', '不良', '信用', '记录', '上报', '统计', '报表', '无', '重大', '安全事故', '发生', '无', '不良', '经营', '行为', '独立', '法人', '企业', '机构'], ['政策', '支持', '平台', '平台', '依托', '单位', '满足', '上述', '条件', '需', '开放', '共享', '原则', '围绕', '高新区', '产业', '共性', '关键性', '技术', '需求', '提供', '公共服务', '支撑', '并经', '西安高新区', '主管部门', '认定', '年审'], ['政策', '支持', '平台', '平台', '依托', '单位', '满足', '上述', '条件

In [16]:
import numpy as np

In [17]:
# 计算相似度
for source_line_item in source_line_list: #源文本每行
    for target_line_item in target_line_list: #目标文本每行
        similarity_mode = 3
        if similarity_mode == 1:  # n_simliarity 方式，#计算两个list列表的Cosine相似度，整体计算
            n_similarity_calced = wv_from_text.n_similarity(source_line_item,target_line_item)
            if n_similarity_calced > 0.8:
                print(source_line_item) #源文本
                print(n_similarity_calced)
                print(target_line_item)#打印目标行
            else:
                pass
        elif similarity_mode == 2: #wmdistance方式，计算两个文档的Word Mover's Distance,整体计算
            wmdistance_calced = wv_from_text.wmdistance(source_line_item, target_line_item)
            if wmdistance_calced < 3.5:
                print(source_line_item) #源文本
                print(target_line_item)#打印目标行
            else:
                pass
        else: #distances方式，逐一单一Word与Word列表中元素的距离，返回列表值
            distances_word_calced = wv_from_text.distances(source_line_item,target_line_item)
#           print(distances_word_calced)
            distances_word_min = np.min(np.array(distances_word_calced)) #求取列表元素最小值
            index_min = (distances_word_calced).tolist().index(min(distances_word_calced)) #获取最小值处索引
            if distances_word_min < 0.3:
                print(source_line_item) #源文本
#                 print(target_line_item[index_min]) #目标文本对应相似元素
                print(distances_word_min)#输出相似度数值
                print(target_line_item)#打印目标行
            else:
                pass

外资
0.0
[]
研发投入
0.297602
['国家级', '高', '新技', '术', '企业', '认定', '且', '年度', '销售收入', '含', '纳入', '西安高新区', '高', '新技', '术', '企业库', '企业', '给予', '一次性', '奖励']
研发投入
0.297602
['国家级', '高', '新技', '术', '企业', '认定', '且', '年度', '销售收入', '含', '纳入', '高新区', '高', '新技', '术', '企业库', '企业', '给予', '一次性', '奖励']
研发投入
0.297602
['国家级', '高', '新技', '术', '企业', '认定', '且', '年度', '销售收入', '含', '纳入', '高新区', '高', '新技', '术', '企业库', '企业', '给予', '一次性', '奖励']
研发投入
0.0
[]
独角兽
1.1920929e-07
[]
总部
0.0
[]
工商关系
-1.1920929e-07
[]
税收关系
-1.1920929e-07
[]
瞪羚企业
0.0
[]
知识产权
0.0
['鼓励', '知识产权', '创造', '运用']
知识产权
0.0
['知识产权', '创造', '奖励']
知识产权
0.0
['申请', '发明专利', '知识产权', '重点', '培育', '企业', '给予', '奖励']
知识产权
0.0
['知识产权', '运用', '奖励']
知识产权
0.0
['支持', '国家', '知识产权', '服务业', '集聚', '发展', '试验区', '知识产权', '运营', '平台', '建设']
知识产权
0.0
['国内外', '品牌', '知识产权', '服务', '机构', '园区', '成立', '独立', '法人', '机构', '给予', '奖励']
知识产权
0.0
['采购', '服务', '方式', '鼓励', '支持', '服务', '机构', '围绕', '知识产权', '密集型', '产业', '开展', '知识产权', '相关', '服务']
知识产权
0.0
['加大', '知识产权', '维权', '援助', '保护', '力度']
知识产权

In [106]:
print(wv_from_text["万人计划"])

[ 0.32256   0.324322  0.43311  -0.165159  0.526738  0.190641 -0.245569
 -0.021141  0.509962 -0.243373  0.506882  0.211813 -0.705412  0.256817
 -0.689561  0.197055 -0.856652 -0.197503 -0.294009  0.251546 -0.455462
 -0.069923 -0.145264  0.513399 -0.449144  0.680086 -0.204233  0.371186
 -0.299719  0.583308  0.015255  0.124017  0.300861  0.590288 -0.124321
 -0.480838  0.454069 -0.042618 -0.416192 -0.344901 -0.126315 -0.086032
  0.051986  0.443776 -0.261137 -0.021561 -0.903307 -0.703411  0.358695
 -0.187988 -0.156977  0.271698 -0.57234  -0.135899 -0.3053   -0.604606
  0.114504  0.818055 -0.337591 -0.393454 -0.267459  0.112595  0.683119
 -0.053355 -0.099466  0.037819  0.284567  0.261493  0.147651  0.317164
 -0.674042  0.116658  0.525118  0.086877 -0.037796  0.113671  0.749925
  0.183775 -0.771791 -0.085236 -0.308281  0.014959 -0.695232  0.12387
 -0.048866  0.273624  0.293923  0.17826  -0.333573 -0.043147  0.430376
 -0.218307  0.204685 -0.038105 -0.247643 -0.572035 -0.207431 -0.059605
 -0.863